## <font color='blue'>Bush School NASA Citizen Science Project: Discover an Exoplanet using the Transit Method!</font>
### Exoplanet Transit Interpretation code (EXOTIC) Modified for Bush School and Jupyter Lab
### <font color='red'>This is YOUR copy </font> of the NASA EXOTIC Jupyter Notebook. 

Run the cells of this notebook in sequence by pressing Shift-Enter. [Here](https://www.youtube.com/watch?v=PwPDttan7KQ) is a screencast that walks you through the process with the sample data (view at high-resolution).  EXOTIC will use our JupyterHub Bushastrolab virtual machine in the cloud to run the code, but it will use images in your JupyterHub account as input. 

Most of the code that is used for the steps in this Jupyter Notebook has been hidden, but if you want to see it, double-click on the corresponding code cell.  (To re-hide Press Shift-Enter.)


### 1. <font color='blue'>Shift-Enter to Select your Assigned Exoplanet below.</font>

In [ ]:
#pip list
import json
import os
myhome=os.getenv('HOME')
import ipywidgets as widgets
from IPython.display import display
radio = widgets.RadioButtons(
    options=['sample_data', 'hatp53b', 'qatar4b', 'wasp135b', 'wasp85ab'],
    layout={'width': 'max-content'}, # If the items' names are long
    description='exoplanet:',
    disabled=False
)
radio

In [ ]:
radio.value

### 2. <font color='blue'>Select Redact below to run on a small dataset for debugging process. The resulting light curve will not be accurate, but it wil help identify issues</font>


In [ ]:
checkbox=widgets.Checkbox(
    value=True,
    description='Redact',
    disabled=False,
    indent=False
)
checkbox

In [ ]:
checkbox.value

In [ ]:
# Opening JSON file
with open('pl_names.json') as json_file:
    nvpair = json.load(json_file)
    # Print the type of data variable
    # print("Type:", type(nvpair))
    exoplanet = radio.value
    redact = checkbox.value
    if (redact):
        exoplanet_image_folder = f'{myhome}/shared/{radio.value}/redact'
        exoplanet_output_folder = f'{myhome}/astronomy/projects/exoplanet_discovery/{radio.value}/redact'
    else:
        exoplanet_image_folder = f'{myhome}/shared/{radio.value}'
        exoplanet_output_folder = f'{myhome}/astronomy/projects/exoplanet_discovery/{radio.value}'

sample_data = False    
if (exoplanet == 'sample_data'):
    sample_data = True;
    exoplanet = 'hatp32b'
    if (redact):
        exoplanet_image_folder = f'{myhome}/shared/HatP32Dec202017/redact'
        exoplanet_output_folder = f'{myhome}/astronomy/projects/exoplanet_discovery/sample-data/redact'
    else:
        exoplanet_image_folder = f'{myhome}/shared/HatP32Dec202017'
        exoplanet_output_folder = f'{myhome}/astronomy/projects/exoplanet_discovery/sample-data'
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OD.png' 
elif (exoplanet == 'hatp53b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374ON.png'
elif (exoplanet == 'kelt23ab'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OP.png'
elif (exoplanet == 'kps1b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OR.png'
elif (exoplanet == 'ngts5b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OS.png'
elif (exoplanet == 'qatar10b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OT.png'
elif (exoplanet == 'qatar4b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OU.png'
elif (exoplanet == 'wasp135b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OV.png'
elif (exoplanet == 'wasp80b'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OW.png'
elif (exoplanet == 'wasp85ab'):
    aavso_exoplanet_starfield = 'https://app.aavso.org/vsp/chart/X28374OX.png'    
else:
    starfield = ''
    
exoplanet_id = nvpair[exoplanet]
# Print the data of dictionary
print(f'redact: {redact}')
print(f'sample_data: {sample_data}')
print(f'Exoplanet: {exoplanet}')
print(f'Exoplanet ID: {exoplanet_id}')
print(f'Exoplanet Transit Image Folder: {exoplanet_image_folder}')
print(f'Exoplanet Transit Output Folder: {exoplanet_output_folder}')
print(f'AAVSO Starfield Image: {aavso_exoplanet_starfield}')

### 3. <font color='blue'>Let's look up your target star and exoplanetary system in the NASA Exoplanet Archive of Caltech. </font>
![exosearch](exosearch.png)
### <font color='red'>Select the top Google search result after running cell below</font>

In [ ]:
try:
	from googlesearch import search
except ImportError:
	print("No module named 'google' found")

# to search
query = f'{exoplanet_id}  NASA exoplanet archive'

for j in search(query, tld="co.in", num=10, stop=10, pause=2):
	print(j)


### 4. Import required Libraries. Please wait until cell finishes executing. You should see '<font color='red'>Planetary Parameters being printed.</font>'

In [ ]:
#@title <font color='red'>Click "run"</font> (at left) to install EXOTIC and give it permission to run on your Google drive files.
#%%capture
#!pip install exotic --upgrade
import bokeh.io
# from exotic.api.plotting import plot_image
from exotic.exotic import NASAExoplanetArchive, get_wcs, find_target
from astropy.time import Time
from barycorrpy import utc_tdb
import numpy as np
from io import BytesIO
from astropy.io import fits
from scipy.ndimage import label
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Viridis256
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, LogTicker
from bokeh.models import BoxZoomTool,WheelZoomTool,ResetTool,HoverTool,PanTool,FreehandDrawTool
from bokeh.io import output_notebook
from bokeh.layouts import row
from pprint import pprint
from IPython.display import Image
from ipywidgets import widgets, HBox
from skimage.transform import rescale, resize, downscale_local_mean
import copy
import matplotlib.pyplot as plt
#!pip install dynesty
import dynesty
from dynesty import plotting
from dynesty.utils import resample_equal
from scipy.stats import gaussian_kde
import os
import re
import json
import subprocess
import PIL
from PIL import Image as Img
from os.path import exists

def fix_planetary_params (p_param_dict):
  for param in p_param_dict.keys():
    if param == "Target Star RA" or param == "Target Star Dec" or param == "Planet Name" or param == "Host Star Name" or param == "Argument of Periastron (deg)":
      continue
    val = p_param_dict[param]
    if val == 0.0 or np.isnan(float(val)):
      if param == "Orbital Eccentricity (0 if null)":
        continue
      if param == "Ratio of Planet to Stellar Radius (Rp/Rs)":
        p_param_dict[param] = 0.151
      if param == "Ratio of Planet to Stellar Radius (Rp/Rs) Uncertainty":
        p_param_dict[param] = 0.151
        if p_param_dict["Host Star Name"] == "Qatar-6":
          p_param_dict[param] = 0.01
      print(f"\nIn the planetary parameters from the NASA Exoplanet Archive, \n\"{param}\" is listed as {val}.\n\n**** This might make EXOTIC crash. ****\n\nIf the parameter is *not* changed below, please edit it\nin the inits file before running EXOTIC.\n")
  p_param_string = json.dumps(p_param_dict)

  planetary_params = "\"planetary_parameters\": {\n"
  num_done, num_total = 0, len(p_param_dict.keys())
  for key, value in p_param_dict.items():
    num_done += 1
    if key == "Target Star RA" or key == "Target Star Dec" or key == "Planet Name" or key == "Host Star Name":
      planetary_params = planetary_params + str(f"    \"{key}\": \"{value}\",\n")
    else:
      if num_done < num_total:
        planetary_params = planetary_params + str(f"    \"{key}\": {value},\n")
      else:
        planetary_params = planetary_params + str(f"    \"{key}\": {value}\n")
  planetary_params = planetary_params + "}"

  return(planetary_params)

##############################################################

#print("YOU NEED TO ENTER THE NAME OF YOUR EXOPLANET TARGET OR ENTER THE SAMPLE TARGET")
#print("CHOICES FOR BUSH EXOPLANET TARGETS ARE 'WASP-80b' OR 'WASP-135 b' OR 'QATAR-4 b' OR 'HAT P-53 b OR ENTER THE SAMPLE TARGET Hat-p-32 b")
#print("DO NOT IGNORE THIS STEP !!!!")

#target=input("Please enter the name of your exoplanet target: ")
target = exoplanet_id
targ = NASAExoplanetArchive(planet=target)
target = targ.planet_info()[0]
planetary_params = ""
if not targ.resolve_name():
  print("Sorry, can't find your target in the Exoplanet Archive.  Unfortunately, this")
  print("isn't going to work until I can find it.  Please re-run this cell, trying")
  print("different formats for your target name, until the target is located.")
  print("Looking it up in the NASA Exoplanet Archive at https://exoplanetarchive.ipac.caltech.edu/")
  print("might help you know where to put the spaces and hyphens and such.")
else:
  p_param_string = targ.planet_info(fancy=True)
  planetary_params = "\"planetary_parameters\": "+p_param_string
  p_param_dict = json.loads(p_param_string)
  planetary_params = fix_planetary_params(p_param_dict)
  print(f"\nNASA Exoplanet Archive planetary parameters for {target}:\n")
  print(planetary_params)


### Please wait until your planetary parameters are read and printed from the NASA Exoplanet Archive - Once compete Shift-Enter:
1. Wait until the cell above is finished running before proceeding.  You will know that it is done running because the Python 3 (ipykernel) 'progress circle' at the top right of the tab turns from filled to unfilled.  (In general, a gray filled (or partially filled) circle means "running", blank circle means "ready for the next step".  Note that there is an incrementing timer plus "headline" command strings scrolling across the status bar at the bottom of the browser window; these are replaced with a total run time and date stamp on completion of the cell. Ypu can also hover over the 'progress circle' at top right to see the time the note book running.  It should be incrementing. If not all of the previous cells are done running.

1. Once it is done, look up details of your target in the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/). You may also use search results at the end of STEP 3 above. In order to fit a lightcurve with appropriate priors, EXOTIC needs to know when the planet is expected to transit and how much starlight the planet is expected to block when it does.  (If you know what an inits file is and you already have one, then you do not *need* to run this target lookup cell, but it is always a good check.)

1. If you are running the sample data, then your target is Hat-p-32 b.  


### 5. Specifying the path to your images for future runs Nothing to do for now - simply Shift-Enter:
* If you are a new user running the Hat-p-32b sample data, just press enter. This will download the sample data to your JupyterLab drive account, ask you to identify the target and pick some comparison stars.  The target star is at the center of [this starchart](https://app.aavso.org/vsp/chart/?star=HAT-P-32&scale=F&orientation=visual&type=chart&fov=18.5&maglimit=18.5&resolution=150&north=down&east=left), labeled with an empty circle.  AAVSO recomends using the numbered stars as comparison stars.  Once you find the stars in your image, EXOTIC will fit a lightcurve and save the output files in your Google drive account. 
* If you are reducing a single transit, enter the path to the google drive folder that holds your images. On your computer in browser
it will look like, for example: My Drive > exotransits > Wasp-43. When asked for the folder, you would enter "exotransits/Wasp-43."  Use the folder icon on the left menu bar to locate the folder, right-click on it, and select "Copy path".  If you have them, calibration images should be in folders named "dark", "flat", and "bias" *within* the folder containing your images.  The first image of the series will be displayed so that you can identify your target and pick some comparison stars (see search results at the end of STEP 3 above for AAVSO recommended comps).  You will also be asked to specify any needed parameters that EXOTIC cannot find from your image headers.
* If you know what an initialization file is and you already have one, put it into the folder with your images and enter the folder name as specified above.  If there is *exactly one* file with a name ending in .json in the folder with your images, this will automatically be used to initialize EXOTIC and reduce your images.
* If you have two or more initialization files and you want to reduce multiple transits, enter the path to a folder that only contains your inits files.  

### 6. Importing Libraries. Reading and Displaying Transit Image for <font color='red'>YOUR INPUT NEEDED FOR REFERENCE STARS</font>

In [ ]:
#@title <font color='red'>Click "run"</font> to start the data reduction.
# If the user presses enter to run the sample data, download sample data if needed and
# put it into a sample-data directory at the top level of the user's Gdrive.  Count
# the .fits files (images) and .json files (inits files) in the directory entered 
# by the user (or in the sample-data directory if the user pressed enter).  If 
# there are at least 20 .fits files, assume this is a directory of images and display
# the first one in the series.  If there is exactly one inits file in the directory, 
# show the specified target and comp coords so that the user can check these against
# the displayed image.  Otherwise, prompt for target / comp coords and make an inits 
# file based  on those (save this new inits file in the folder with the output files 
# so that the student can consult it later).  Finally, run EXOTIC with the newly-made 
# or pre-existing inits file, plus any other inits files in the directory.

#########################################################
def display_image(filename):
    print(f"{filename}")
    hdu = fits.open(filename)

# replace 0's with extension      # Stuff to put in if the hdul ever becomes a problem.    
#    extension = 0
#    image_header = hdul[extension].header
#    while image_header["NAXIS"] == 0:
#      extension += 1
#      image_header = hdul[extension].header

    dheader = dict(hdu[0].header)
  
    data = hdu[0].data
    megapixel_factor = (data.shape[0])*(data.shape[1])/1000000.0
    if megapixel_factor > 5:
      print(f"Downsampling image because it has {megapixel_factor} megapixels.")
      image_downscaled = downscale_local_mean(data, (2, 2)).astype(int)
      data = image_downscaled
    
    max_y = len(data)
    max_x = len(data[0])
    p_height = 500
    p_width = int((p_height/max_y) * max_x)

    # quick hot pixel/ cosmic ray mask
    # mask, cdata = detect_cosmics(
    #     data, psfmodel='gauss',
    #     psffwhm=4, psfsize=2*round(4)+1, # just a guess
    #     sepmed=False, sigclip = 4.25,
    #     niter=3, objlim=10, cleantype='idw', verbose=False
    # )

    # create a figure with text on mouse hover
    fig = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_width=p_width, plot_height=p_height,
        tools=[PanTool(),BoxZoomTool(),WheelZoomTool(),ResetTool(),HoverTool()])
    fig.x_range.range_padding = fig.y_range.range_padding = 0

    r = fig.multi_line('x', 'y', source={'x':[],'y':[]},color='white',line_width=3)
    fig.add_tools(FreehandDrawTool(renderers=[r]))

    # set up a colobar + data range
    color_mapper = LogColorMapper(palette="Cividis256", low=np.percentile(data, 55), high=np.percentile(data, 99))

    # must give a vector of image data for image parameter
    fig.image(
        image=[data],
          x=0, y=0, dw=hdu[0].data.shape[1], dh=hdu[0].data.shape[0],
          level="image", color_mapper=color_mapper
    )
    fig.grid.grid_line_width = 0.5

    color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                         label_standoff=12, border_line_color=None, location=(0,0))

    fig.add_layout(color_bar, 'right')

    if sample_data:
        starfield = figure()
        starfield.image_url(['https://app.aavso.org/vsp/chart/X28194FDL.png'], 0, 1, 0.8, h=0.6)  
        show(row(fig,starfield))
    else:
        starfield = figure()
        starfield.image_url([aavso_exoplanet_starfield], 0, 1, 0.8, h=0.6)  
        show(row(fig,starfield))
    
#########################################################

def floats_to_ints(l):
  while (True):
#    print (l)
    m = re.search(r"^(.*?)(\d+\.\d+)(.*?)$", l)
    if m:
      start, fl, end = m.group(1), float(m.group(2)), m.group(3)
      l = start+str("%.0f" % fl)+end
    else:
      return(l)
  
#########################################################

# Find a field in the image fits header or prompt the user to enter the corresponding
# value.

def check_dir(p):
  p = p.replace("\\", "/")

  if not(os.path.isdir(p)):
    print(f"Problem: the directory {p} doesn't seem to exist")
    print("on your JupyterLab filesystem.")
    return("")
  return(p)

#########################################################

def add_sign(var):
  str_var = str(var)
  m=re.search(r"^[\+\-]", str_var)
  if m:
    return(str_var)
  if float(var) >= 0:
    return(str("+%.6f" % float(var)))
  else:
    return(str("-%.6f" % float(var)))

#########################################################

def get_val(hdr, ks):
  for key in ks:
    if key in hdr.keys():
      return hdr[key]
    if key.lower() in hdr.keys():
      return hdr[key.lower()]
    new_key = key[0]+key[1:len(key)].lower()  # first letter capitalized
    if new_key in hdr.keys():
      return hdr[new_key]
  return("")

#########################################################

def process_lat_long(val, key):
  m = re.search(r"\'?([+-]?\d+)[\s\:](\d+)[\s\:](\d+\.?\d*)", val)
  if m:
    deg, min, sec = float(m.group(1)), float(m.group(2)), float(m.group(3))
    if deg < 0:
      v = deg - (((60*min) + sec)/3600)
    else:
      v = deg + (((60*min) + sec)/3600)
    return(add_sign(v))
  m = re.search("^\'?([+-]?\d+\.\d+)", val)
  if m:
    v = float(m.group(1))
    return(add_sign(v))
  else:
    print(f"Cannot match value {val}, which is meant to be {key}.")

#########################################################

# Convert a MicroObservatory timestamp (which is in local time) to UTC.

def convert_Mobs_to_utc(datestamp, latitude, longitude, height):

#  print(datestamp)
  t = Time(datestamp[0:21], format='isot', scale='utc')
  t -= 0.33

  return(str(t)[0:10])

#########################################################

def find (hdr, ks, obs):
  # Special stuff for MObs and Boyce-Astro Observatories
  boyce = {"FILTER": "ip", "LATITUDE": "+32.6135", "LONGITUD": "-116.3334", "HEIGHT": 1405 }
  mobs = {"FILTER": "V", "LATITUDE": "+37.04", "LONGITUD": "-110.73", "HEIGHT": 2606 }

  if "OBSERVAT" in hdr.keys() and hdr["OBSERVAT"] == 'Whipple Observatory':
    obs = "MObs"

#  if "USERID" in hdr.keys() and hdr["USERID"] == 'PatBoyce':
#    obs = "Boyce"

  if obs == "Boyce":
    boyce_val = get_val(boyce, ks)
    if (boyce_val != ""):
      return(boyce_val)
  if obs == "MObs":
    mobs_val = get_val(mobs, ks)
    if (mobs_val != ""):
      return(mobs_val)

  val = get_val(hdr, ks)

  if ks[0] == "LATITUDE" and val != "":         # Because EXOTIC needs these with signs shown.
    return(process_lat_long(str(val), "latitude"))
  if ks[0] == "LONGITUD" and val != "":
    return(process_lat_long(str(val), "longitude"))

  if (val != ""):
    return(val)

  print(f"\nI cannot find a field with any of these names in your image header: \n{ks}.")
  print("Please enter the value (not the name of the header field, the actual value) that should")
  print("be used for the value associated with this field.\n")
  if ks[0] == "HEIGHT":
    print("The units of elevation are meters.")
  value = input("")
  return(value)

###############################################

def look_for_calibration(image_dir):
  darks_dir, flats_dir, biases_dir = "null", "null", "null"

  m = re.search(r"(.*?)(\d\d\d\d\-\d\d\-\d\d)\/images", image_dir)  # This handles the way I set up the MObs image paths for my seminar teams.
  if m:
    prefix, date = m.group(1), m.group(2)
    darks = prefix+date+"/darks"
    if os.path.isdir(darks):
      darks_dir = str("\""+darks+"\"")
      
  d_names = ["dark", "darks", "DARK", "DARKS", "Dark", "Darks"]  # Possible names for calibration image directories.
  f_names = ["flat", "flats", "FLAT", "FLATS", "Flat", "Flats"]
  b_names = ["bias", "biases", "BIAS", "BIASES", "Bias", "Biases"]

  for d in d_names:
    if os.path.isdir(os.path.join(image_dir, d)):
      darks_dir = str("\""+os.path.join(image_dir, d)+"\"")
      break

  for f in f_names:
    if os.path.isdir(os.path.join(image_dir, f)):
      flats_dir = str("\""+os.path.join(image_dir, f)+"\"")
      break

  for b in b_names:
    if os.path.isdir(os.path.join(image_dir, b)):
      biases_dir = str("\""+os.path.join(image_dir, b)+"\"")
      break

  return(darks_dir, flats_dir, biases_dir)

###############################################

# Writes a new inits file into the directory with the output plots.  This prompts
# for needed information that it cannot find in the fits header of the first image.

def make_inits_file(planetary_params, image_dir, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data):
  inits_file_path = output_dir+"inits.json"
  hdul = fits.open(first_image)
  hdr = dict(hdul[0].header)

  min, max = "null", "null"
  filter = find(hdr, ['FILTER', 'FILT'], obs)
  if filter == "w":
    filter = "PanSTARRS-w"
    min = "404"
    max = "846"
  if filter == "Clear":
    filter = "V"
  if filter == "ip":
    min = "690"
    max = "819"
  if filter == "EXO":
    filter = "CBB"
  if re.search(r"Green", filter, re.IGNORECASE):
    filter = "SG"
    
  date_obs = find(hdr,["DATE", "DATE_OBS", "DATE-OBS"], obs)
  date_obs = date_obs.replace("/", "_")
  longitude = find(hdr,['SITELONG', 'LONGITUD', 'LONG', 'LONGITUDE'],obs)
  latitude = find(hdr,['SITELAT', 'LATITUDE', 'LAT'],obs)
  height = int(find(hdr, ['HEIGHT', 'ELEVATION', 'ELE', 'EL', 'OBSGEO-H', 'ALT-OBS', 'SITEELEV'], obs))
  obs_notes = "N/A"
  sec_obs_code = "N/A"

  mobs_data = False
  # For MObs, the date is local rather than UTC, so convert.
  if "OBSERVAT" in hdr.keys() and hdr["OBSERVAT"] == 'Whipple Observatory':
    date_obs = convert_Mobs_to_utc(date_obs, latitude, longitude, height)
    weather = hdr["WEATHER"] 
    temps = float(hdr["TELTEMP"]) - float(hdr["CAMTEMP"])
    obs_notes = str("First image seeing %s (0: poor, 100: excellent), Teltemp - Camtemp %.1f.  These observations were conducted with MicroObservatory, a robotic telescope network managed by the Harvard-Smithsonian Center for Astrophysics on behalf of NASA's Universe of Learning. This work is supported by NASA under award number NNX16AC65A to the Space Telescope Science Institute." % (weather, temps))
    sec_obs_code = "MOBS"  
    mobs_data = True
  
  if aavso_obs_code == "":
      aavso_obs_code = "N/A"

  obs_date = date_obs[0:10]
  (darks_dir, flats_dir, biases_dir) = look_for_calibration(image_dir)

  with open(inits_file_path, 'w') as inits_file:
    inits_file.write("""
{
  %s,
    "user_info": {
            "Directory with FITS files": "%s",
            "Directory to Save Plots": "%s",
            "Directory of Flats": %s,
            "Directory of Darks": %s,
            "Directory of Biases": %s,

            "AAVSO Observer Code (N/A if none)": "%s",
            "Secondary Observer Codes (N/A if none)": "%s",

            "Observation date": "%s",
            "Obs. Latitude": "%s",
            "Obs. Longitude": "%s",
            "Obs. Elevation (meters)": %d,
            "Camera Type (CCD or DSLR)": "CCD",
            "Pixel Binning": "1x1",
            "Filter Name (aavso.org/filters)": "%s",
            "Observing Notes": "%s",

            "Plate Solution? (y/n)": "y",
            "Align Images? (y/n)": "y",

            "Target Star X & Y Pixel": %s,
            "Comparison Star(s) X & Y Pixel": %s
    },    
    "optional_info": {
            "Pixel Scale (Ex: 5.21 arcsecs/pixel)": null,
            "Filter Minimum Wavelength (nm)": %s,
            "Filter Maximum Wavelength (nm)": %s
    }
}
""" % (planetary_params, image_dir, output_dir, flats_dir, darks_dir, biases_dir, 
       aavso_obs_code, sec_obs_code, obs_date, latitude, longitude, height, filter, 
       obs_notes, targ_coords, comp_coords, min, max))

  print("\nWithin your folder of images, there is now a new folder called EXOTIC_output.")
  print("This folder contains an initialization file for EXOTIC called inits.json.")
  print("The same folder will contain the output files and images when EXOTIC finishes running.")

  if not mobs_data:  
    print(f"\nThe inits.json file currently says that your observatory latitude was {latitude} deg,")
    print(f"longitude was {longitude} deg, and elevation was {height}m.  \n")
    print("*** If any of these are incorrect, please change them in the inits.json file. ***")
    print("*** (Please make sure that Western longitudes have a negative sign! ***")
    print("*** TheSkyX sometimes stamps Western longitudes as positive; this needs to be switched! ***\n")

  print("\nIf you want to change anything in the inits file, please do that now.")
  print("When you are done, press enter to continue.")
  okay = input()

  return(inits_file_path)

#########################################################
bokeh.io.output_notebook()
!mkdir -p {output_dir}
p = check_dir(exoplanet_image_folder)
output_dir = os.path.join(exoplanet_output_folder, "EXOTIC_output/")    
print(output_dir)
!mkdir -p {output_dir}
print(sample_data)
print(aavso_exoplanet_starfield)

inits = []    # array of paths to any inits files found in the directory
files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]
fits_count, first_image = 0, ""
files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]
fits_count, first_image = 0, ""
for f in files:
  if re.search(r"\.f[itz]+s?\.?g?z?$", f, re.IGNORECASE):
    if first_image == "":
      first_image = os.path.join(p, f)
    fits_count += 1
  if re.search(r"\.json$", f, re.IGNORECASE):
    inits.append(os.path.join(p, f))
print(f"Found {fits_count} image files and {len(inits)} initialization files in the directory.")

hdul = fits.open(first_image)
hdr = dict(hdul[0].header)
#print(hdr)

if fits_count >= 19:                  # more than 20 images in the folder --> run EXOTIC on these images
  if len(inits) == 1:                 # one inits file exists
    inits_path = os.path.join(p, inits[0])
    with open(inits_path) as i_file:
      inits_data = i_file.read()
      d = json.loads(inits_data)
      targ_coords = d["user_info"]["Target Star X & Y Pixel"]
      comp_coords = d["user_info"]["Comparison Star(s) X & Y Pixel"]
      input_dir = d["user_info"]["Directory with FITS files"]
      if input_dir != p:
        print(f"The directory with fits files should be {p} but your inits file says {input_dir}.")
        print("This may or may not cause problems.  Just letting you know.")
      print(f"Coordinates from your inits file:\ntarget: {targ_coords}\ncomps: {comp_coords}")
      output_dir = d["user_info"]["Directory to Save Plots"]
      
  else:                               # no inits file: display image and prompt for target, comp coords;
                                      # then make an inits file and put it into the output directory (with the plots)
    print("There are either 0 or > 1 inits files in your image directory, so we'll make a new one.")
    print("Displaying first image (this can take a minute).")
    display_image(first_image)
    obs = ""
    print("Enter the coordinates of the target in the format [x, y]")
    print("Do NOT include decimals in the coordinates.")
    print("Be sure to include the square brackets and comma.")
    print("Target Coordinates: [x, y]")
    targ_coords = input()  
    print("\nEnter the coordinates of your comparison stars in the format [[x1, y1], [x2, y2]]")
    print("Be sure to include all square brackets and commas.")
    print("Do NOT include decimals in the coordinates.")    
    print("(E.g. if you only have one comp, you would enter [[x, y]])")
    print("Comparison Star Coordinates: [[x1, y1], [x2, y2]]")
    comp_coords = input()
    aavso_obs_code = ""
#    if not sample_data:
#      print("If you have an AAVSO Observer code, enter it here.")
#      print("If not (or if you are not sure), just press enter.")
#      aavso_obs_code = input()
#    print("If you want an alternate output directory, type it here.")
#    alt_out = input()
#    if (re.search(r"\w", alt_out)):
#      output_dir = alt_out
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
    inits = [make_inits_file(planetary_params, p, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data)]

  if not os.path.isdir(output_dir):    # Make the output directory if it does not exist already.
    os.mkdir(output_dir)
  output_dir_for_shell = output_dir.replace(" ", "\ ")

# At this point, we have made an inits file if needed and are ready to run it.
# Or, if there were < 20 images in the directory (meaning that the directory was 
# only contained inits files, we'll run those one-by-one.

print("Path to the inits file(s) that will be used:")

for inits_file in inits:
  print(inits_file)

num_inits = len(inits)

### 7. <font color='blue'> The bulk of the data processing happens below simply Shift-Enter the next cell. It will take anywhere from 15 minutes to 1 hour
    
####    <font color='red'>You will know that it is done running because the Python 3 (ipykernel) 'progress circle' at the top right of the tab turns from filled to unfilled.  (In general, a gray filled (or partially filled) circle means "running", blank circle means "ready for the next step".  Note that there is an incrementing timer plus "headline" command strings scrolling across the status bar at the bottom of the browser window; these are replaced with a total run time and date stamp on completion of the cell. Ypu can also hover over the 'progress circle' at top right to see the time the note book running.  It should be incrementing.

In [ ]:
commands = []
for inits_file in inits:
  with open(inits_file) as i_file:
    inits_data = i_file.read()
    d = json.loads(inits_data)
    date_obs = d["user_info"]["Observation date"]
    planet = d["planetary_parameters"]["Planet Name"]
    output_dir = d["user_info"]["Directory to Save Plots"]
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
    inits_file_for_shell = inits_file.replace(" ", "\\ ")
    run_exotic = str(f"exotic -red {inits_file_for_shell} -ov < no.txt")
    debug_exotic_run = str(f"!exotic -red \"{inits_file}\" -ov < no.txt")

    commands.append({"inits_file_for_shell": inits_file_for_shell, "output_dir": output_dir, 
                      "planet": planet, "date_obs": date_obs, 
                      "run_exotic": run_exotic, "debug_exotic_run": debug_exotic_run
                      })
    print(f"{debug_exotic_run}")
    !eval "$run_exotic"

    lightcurve = os.path.join(output_dir,"FinalLightCurve_"+planet+"_"+date_obs+".png")
    fov = os.path.join(output_dir,"temp/FOV_"+planet+"_"+date_obs+"_LinearStretch.png")
    triangle = os.path.join(output_dir,"temp/Triangle_"+planet+"_"+date_obs+".png")
    print(f"lightcurve: {lightcurve}\nfov: {fov}\ntriangle: {triangle}\n")

    if not (os.path.isfile(lightcurve) and os.path.isfile(fov) and os.path.isfile(triangle)):
      print(f"Something went wrong with {planet} {date_obs}.\nCopy the command below into a new cell and run to find the error:\n{debug_exotic_run}\n")
      continue

    imageA = widgets.Image(value=open(lightcurve, 'rb').read())
    imageB = widgets.Image(value=open(fov, 'rb').read())
    hbox = HBox([imageB, imageA])
    display(hbox)
    display(Image(filename=triangle))
    


## <font color='red'>  The cell above will take between 15 minutes and an hour to run after you enter your path, please be patient, leave the notebook running.  See Updates being printed ABOVE THIS LINE If you see an error message, we need to debug our issue.  

## <font color='blue'> In the meanwhile, you can do other tasks on your laptop. When you see the Planetary Transit Light Curve - You are Done.  Now we can study the images and make conclusions!!

**Explanation of the plots that are displayed at the end of the run:**

The plot titled "FOV" is the "Field of View" of the images.  Your target is circled, and if EXOTIC used a comparison star, that is circled too.  (EXOTIC does a quick preliminary fit using each of your comps, and chooses the one that gives the cleanest lightcurve to use in the more time-consuming, more rigorous fitting routine.  Sometimes, though, the cleanest lightcurve results from not using *any* comps.  In that case, you will only see your target circled.)

The lightcurve shows the dip in light as the planet blocked the star during your observations.  The last plot shows what are called the "posteriors" of the fit, obtained by an algorithm that is called "nested sampling".  This shows how certain each of the parameters are.  The main parameter of interest is the transit midpoint, tmid.  You should see Gaussian distributions; if you don't, then it might be because the data were noisy.  

All of the graphs above, plus several other output files, are now saved to your output directory (if you used this Colaboratory to make the inits file, that is in the EXOTIC_output folder within the folder of images on your Google drive).  The textfile whose name starts with "AAVSO" is already in the correct format to be uploaded to the American Association of Variable Star Observers' [Exoplanet Database](https://www.aavso.org/apps/exosite/doc).  Uploading will allow other astronomers to use your data to plan their observing campaigns for your target, and will allow NASA to use your data to optimize space telescopes.  (However, please do not upload the sample data to AAVSO; that has already been done.)


**Note that you can use this same Jupyter Notebok again to process your next transit!**

